In [1]:
import cv2
import numpy as np
import pytesseract

In [8]:
def load_image(file_path):
    image = cv2.imread(file_path)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [9]:
def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    gray = clahe.apply(gray)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    return blurred

In [3]:
def process_image(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Detect edges using Canny
    edges = cv2.Canny(blurred, 50, 150)
    
    # Find contours in the image
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    card_images = []
    
    for contour in contours:
        # Approximate contour to polygon
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        # If the contour has four vertices, assume it's a rectangle (card)
        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(approx)
            # Check aspect ratio to filter out non-card contours
            if 0.5 <= w/h <= 1.5:
                card_image = image[y:y+h, x:x+w]
                card_images.append(card_image)
    
    return card_images

In [20]:
def recognize_cards(image):
    if len(image.shape) == 3:  # Jeśli obraz jest kolorowy
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:  # Jeśli obraz jest już w skali szarości
        gray = image

    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Zwiększ kontrast
    kernel = np.ones((2,2),np.uint8)
    thresh = cv2.dilate(thresh, kernel, iterations=1)
    
    # Konfiguracja Tesseract dla pojedynczych znaków
    custom_config = r'--oem 3 --psm 10 -c tessedit_char_whitelist=AKQJ1098765432'
    text = pytesseract.image_to_string(thresh, config=custom_config)
    
    # Oczyszczenie wyniku
    text = ''.join(c for c in text if c.isalnum())
    return text

In [18]:

def main(file_path):
    image = load_image(file_path)
    preprocessed = preprocess_image(image)
    card_images = detect_cards(preprocessed)
    
    for i, card_image in enumerate(card_images):
        card = recognize_cards(card_image)
        print(f"Card {i+1}: {card}")

    # Wyświetl wykryte karty (do debugowania)
    for i, card in enumerate(card_images):
        cv2.imshow(f"Card {i+1}", card)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [11]:
def detect_cards(image):
    edges = cv2.Canny(image, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    card_images = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < 1000 or area > 100000:  # Filter out small and large contours
            continue
        
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        if len(approx) == 4:
            pts = np.float32(approx)
            rect = cv2.minAreaRect(contour)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            
            width = int(rect[1][0])
            height = int(rect[1][1])
            
            dst_pts = np.array([[0, height-1],
                                [0, 0],
                                [width-1, 0],
                                [width-1, height-1]], dtype="float32")
            
            M = cv2.getPerspectiveTransform(pts, dst_pts)
            warped = cv2.warpPerspective(image, M, (width, height))
            
            card_images.append(warped)
    
    return card_images

In [ ]:
# Example usage
file_path = "D:\\sierra bots\\poker test.png"
main(file_path)

Card 1: 2
Card 2: 22
Card 3: A
Card 4: 2
